[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_22/uebung_neuronale_netze.ipynb)

# Übung: Neuronale Netze mit MNIST Dataset
## MLP vs. CNN Vergleich

In dieser Übung schauen wir uns zwei Arten von neuronalen Netzen an:
- **Multi-Layer Perceptron (MLP)**: Klassisches neuronales Netz mit vollverbundenen Schichten
- **Convolutional Neural Network (CNN)**: Speziell für Bilddaten entwickelt

Wir verwenden den **MNIST Dataset**: 28×28 Pixel Bilder handgeschriebener Ziffern (0-9) - der Klassiker für Computer Vision!

## 1. Import Required Libraries

In [ ]:
# Standard Libraries
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow/Keras für Daten und beide Modelle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Scikit-learn nur für Metriken
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Bibliotheken geladen!")
print(f"TensorFlow Version: {tf.__version__}")

# Reproducibility
tf.random.set_seed(42)
np.random.seed(42)

## 2. Load and Explore the MNIST Dataset

In [ ]:
# MNIST Dataset laden
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

print("Dataset Informationen:")
print(f"Training Bilder: {X_train_full.shape[0]}")
print(f"Test Bilder: {X_test.shape[0]}")
print(f"Bildgröße: {X_train_full.shape[1]}×{X_train_full.shape[2]} Pixel")
print(f"Anzahl Klassen: {len(np.unique(y_train_full))}")
print(f"Klassen: {np.unique(y_train_full)}")
print(f"Pixel-Werte: {X_train_full.min()} bis {X_train_full.max()}")

# Für schnelleres Training: nur Subset verwenden
subset_size = 10000  # Statt 60000
indices = np.random.choice(len(X_train_full), subset_size, replace=False)
X_train = X_train_full[indices]
y_train = y_train_full[indices]

print(f"\nVerwendet für Training: {len(X_train)} Bilder")

# Einige Beispielbilder anzeigen
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[i], cmap='gray')
    ax.set_title(f'Label: {y_train[i]}')
    ax.axis('off')
plt.suptitle('Beispielbilder aus dem MNIST Dataset')
plt.tight_layout()
plt.show()

## 3. Data Preprocessing

In [ ]:
# Daten normalisieren
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

print(f"Training set: {X_train.shape[0]} Bilder")
print(f"Test set: {X_test.shape[0]} Bilder")

# Daten für MLP: flach machen (28×28 → 784)
X_train_mlp = X_train.reshape(-1, 28*28)
X_test_mlp = X_test.reshape(-1, 28*28)

# Daten für CNN: Channel-Dimension hinzufügen
X_train_cnn = X_train.reshape(-1, 28, 28, 1)
X_test_cnn = X_test.reshape(-1, 28, 28, 1)

print(f"\nMLP Input Shape: {X_train_mlp.shape}")
print(f"CNN Input Shape: {X_train_cnn.shape}")

# Test-Subset für schnellere Evaluation
test_subset_size = 2000
test_indices = np.random.choice(len(X_test), test_subset_size, replace=False)
X_test_mlp = X_test_mlp[test_indices]
X_test_cnn = X_test_cnn[test_indices]
y_test = y_test[test_indices]

print(f"Test subset für Evaluation: {len(X_test_mlp)} Bilder")

## 4. Multi-Layer Perceptron (MLP) Implementation

In [ ]:
# MLP mit Keras (für Konsistenz)
print("Aufbau MLP Modell...")

mlp_model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

mlp_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("MLP Architektur:")
mlp_model.summary()

In [ ]:
# MLP trainieren
print("Training MLP...")
mlp_history = mlp_model.fit(
    X_train_mlp, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.1,
    verbose=1
)

# MLP evaluieren
mlp_loss, mlp_accuracy = mlp_model.evaluate(X_test_mlp, y_test, verbose=0)
print(f"\nMLP Test Genauigkeit: {mlp_accuracy:.4f}")

# Vorhersagen für späteren Vergleich
y_pred_mlp = np.argmax(mlp_model.predict(X_test_mlp, verbose=0), axis=1)

## 5. Convolutional Neural Network (CNN) Implementation

In [ ]:
# CNN mit Keras
print("Aufbau CNN Modell...")

cnn_model = keras.Sequential([
    # Erste Convolutional Block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(2, 2),
    
    # Zweite Convolutional Block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    
    # Dritte Convolutional Block
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    # Flatten und Dense Layers
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("CNN Architektur:")
cnn_model.summary()

In [ ]:
# CNN trainieren
print("Training CNN...")
cnn_history = cnn_model.fit(
    X_train_cnn, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.1,
    verbose=1
)

# CNN evaluieren
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_cnn, y_test, verbose=0)
print(f"\nCNN Test Genauigkeit: {cnn_accuracy:.4f}")

# Vorhersagen für Vergleich
y_pred_cnn = np.argmax(cnn_model.predict(X_test_cnn, verbose=0), axis=1)

## 6. Model Comparison and Evaluation

In [ ]:
# Vergleich der Modelle
print("=== MODELL VERGLEICH ===")
print(f"MLP Genauigkeit:  {mlp_accuracy:.4f}")
print(f"CNN Genauigkeit:  {cnn_accuracy:.4f}")
print(f"Verbesserung:     {cnn_accuracy - mlp_accuracy:.4f}")

# Parameter-Anzahl vergleichen
mlp_params = mlp_model.count_params()
cnn_params = cnn_model.count_params()
print(f"\nMLP Parameter:    {mlp_params:,}")
print(f"CNN Parameter:    {cnn_params:,}")
print(f"Verhältnis CNN/MLP: {cnn_params/mlp_params:.1f}×")

# Training History Visualisierung
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(mlp_history.history['loss'], label='MLP Training')
plt.plot(mlp_history.history['val_loss'], label='MLP Validation')
plt.plot(cnn_history.history['loss'], label='CNN Training')
plt.plot(cnn_history.history['val_loss'], label='CNN Validation')
plt.title('Training Loss Vergleich')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(mlp_history.history['accuracy'], label='MLP Training')
plt.plot(mlp_history.history['val_accuracy'], label='MLP Validation')
plt.plot(cnn_history.history['accuracy'], label='CNN Training')
plt.plot(cnn_history.history['val_accuracy'], label='CNN Validation')
plt.title('Training Accuracy Vergleich')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 3, 3)
models = ['MLP', 'CNN']
accuracies = [mlp_accuracy, cnn_accuracy]
colors = ['skyblue', 'lightcoral']
plt.bar(models, accuracies, color=colors)
plt.title('Test Accuracy Vergleich')
plt.ylabel('Accuracy')
plt.ylim(0.9, 1.0)
for i, acc in enumerate(accuracies):
    plt.text(i, acc + 0.002, f'{acc:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Konfusionsmatrizen vergleichen
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# MLP Konfusionsmatrix
cm_mlp = confusion_matrix(y_test, y_pred_mlp)
im1 = axes[0].imshow(cm_mlp, interpolation='nearest', cmap='Blues')
axes[0].set_title(f'MLP Confusion Matrix\nAccuracy: {mlp_accuracy:.3f}')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')
axes[0].set_xticks(range(10))
axes[0].set_yticks(range(10))

# CNN Konfusionsmatrix
cm_cnn = confusion_matrix(y_test, y_pred_cnn)
im2 = axes[1].imshow(cm_cnn, interpolation='nearest', cmap='Blues')
axes[1].set_title(f'CNN Confusion Matrix\nAccuracy: {cnn_accuracy:.3f}')
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')
axes[1].set_xticks(range(10))
axes[1].set_yticks(range(10))

# Zahlen in die Matrizen schreiben (nur bei kleinen Werten für Lesbarkeit)
for i in range(10):
    for j in range(10):
        if cm_mlp[i, j] > 0:
            axes[0].text(j, i, str(cm_mlp[i, j]), ha='center', va='center', 
                        color='white' if cm_mlp[i, j] > cm_mlp.max()/2 else 'black', fontsize=8)
        if cm_cnn[i, j] > 0:
            axes[1].text(j, i, str(cm_cnn[i, j]), ha='center', va='center',
                        color='white' if cm_cnn[i, j] > cm_cnn.max()/2 else 'black', fontsize=8)

plt.colorbar(im1, ax=axes[0])
plt.colorbar(im2, ax=axes[1])
plt.tight_layout()
plt.show()

In [ ]:
# Beispiel-Vorhersagen visualisieren
fig, axes = plt.subplots(3, 8, figsize=(16, 8))

# Nehme 8 zufällige Test-Beispiele
np.random.seed(42)
sample_indices = np.random.choice(len(X_test_cnn), 8, replace=False)

for i, idx in enumerate(sample_indices):
    # Originalbild
    axes[0, i].imshow(X_test_cnn[idx].reshape(28, 28), cmap='gray')
    axes[0, i].set_title(f'True: {y_test[idx]}', fontsize=10)
    axes[0, i].axis('off')
    
    # MLP Vorhersage
    mlp_pred = y_pred_mlp[idx]
    mlp_color = 'green' if mlp_pred == y_test[idx] else 'red'
    axes[1, i].text(0.5, 0.5, f'MLP\n{mlp_pred}', transform=axes[1, i].transAxes,
                   ha='center', va='center', fontsize=14, color=mlp_color, weight='bold')
    axes[1, i].axis('off')
    
    # CNN Vorhersage
    cnn_pred = y_pred_cnn[idx]
    cnn_color = 'green' if cnn_pred == y_test[idx] else 'red'
    axes[2, i].text(0.5, 0.5, f'CNN\n{cnn_pred}', transform=axes[2, i].transAxes,
                   ha='center', va='center', fontsize=14, color=cnn_color, weight='bold')
    axes[2, i].axis('off')

axes[0, 0].set_ylabel('Original', fontsize=12)
axes[1, 0].set_ylabel('MLP Pred.', fontsize=12)
axes[2, 0].set_ylabel('CNN Pred.', fontsize=12)
plt.suptitle('Beispiel-Vorhersagen: Grün = Richtig, Rot = Falsch', fontsize=14)
plt.tight_layout()
plt.show()

# Fehleranalyse: Wo macht das CNN weniger Fehler als das MLP?
mlp_errors = (y_pred_mlp != y_test)
cnn_errors = (y_pred_cnn != y_test)
cnn_better = mlp_errors & ~cnn_errors  # MLP falsch, CNN richtig

print(f"\nFehleranalyse:")
print(f"MLP Fehler: {mlp_errors.sum()}/{len(y_test)} ({100*mlp_errors.mean():.1f}%)")
print(f"CNN Fehler: {cnn_errors.sum()}/{len(y_test)} ({100*cnn_errors.mean():.1f}%)")
print(f"CNN besser als MLP: {cnn_better.sum()} Fälle")

## 🎯 Fazit und Erkenntnisse

**Wichtige Beobachtungen:**

1. **CNNs sind deutlich besser für Bilder**: Bei MNIST (28×28) zeigt sich der CNN-Vorteil klar - typisch 2-4% bessere Accuracy als MLPs.

2. **Parameter-Effizienz bei größeren Bildern**: Obwohl das CNN hier mehr Parameter hat, skaliert es bei noch größeren Bildern besser als MLPs.

3. **Räumliche Strukturerkennung**: CNNs erkennen lokale Muster (Striche, Kurven) und kombinieren sie zu komplexeren Formen - perfekt für handgeschriebene Ziffern.

4. **Robustheit**: CNNs sind oft robuster gegen Verschiebungen und kleine Verzerrungen der Bilder.

**Typische Ergebnisse:**
- **MLP auf MNIST:** ~97-98% Accuracy
- **CNN auf MNIST:** ~99%+ Accuracy

**Wann welches Modell?**
- **MLP**: Tabellarische Daten, kleine Bilder, schnelle Prototypen
- **CNN**: Bilder aller Größen, Computer Vision, wenn räumliche Struktur wichtig ist

**Der Durchbruch:** Erst bei "echten" Bildgrößen (28×28 und größer) zeigen CNNs ihre wahre Stärke. Bei winzigen 8×8 Bildern können MLPs noch mithalten.

**Zum Experimentieren:**
- Verändere die CNN-Architektur (mehr/weniger Filter, andere Kernel-Größen)
- Probiere andere Optimierer aus (SGD mit Momentum)
- Teste mit noch größeren Datensätzen (CIFAR-10, ImageNet)
- Implementiere Data Augmentation (Rotation, Translation)